# Step 1 : merge desirability_by_school with postcodes_by_urn
The point of this step is to have desirability by postcodes. 
There are no postcodes in the data used in the process of creating desirability. However, there is a designated postcode for each school, each UNR is specific to the school and can be associated to a postcode.
The point of this step is to use a table that associates each unr to a postcode to have a postcode entry in the desirability

In [2]:
import numpy as np
import pandas as pd

In [3]:
desirability_by_school = pd.read_csv('data/desirability_by_school.csv')
postcodes_by_urn = pd.read_csv('data/urn_postcodes_2014.csv')

In [4]:
old_length = len(desirability_by_school)
old_urn = desirability_by_school['URN']


desirability_by_school = desirability_by_school.merge(postcodes_by_urn, on='URN', copy=False)


new_length = len(desirability_by_school)
new_urn = desirability_by_school['URN']

print("Dropped " + str(old_length - new_length) + " our of " + str(old_length) + " unrecorded UNR in the process")
# uncomment to print the difference
# pd.concat([old_urn,new_urn]).drop_duplicates(keep=False)


Dropped 47 our of 9748 unrecorded UNR in the process


# Step 2 : merge desirability_by_school with postcodes
The point of this step is to add the corresponding MSOA to desirability_by_school by computing it from the postcodes using the corresponding "postcode to MSOA" table.


This has to be done by using chunk, right outer joins and concatenation.
This is because "postcodes.csv" is very large (several Gb). It can't be stored in memory. It won't be a problem in the final result because very few postcode actually appear in our dataset.

In [10]:
final_desirability = pd.DataFrame()              
chunksize = 10 ** 6
for chunk in pd.read_csv('data/postcodes.csv', chunksize=chunksize,low_memory=False):
    final_desirability = pd.concat([final_desirability, desirability_by_school.merge(chunk, how='inner', on='Postcode', copy=False)])

Dropped 9636 out of 9701 unrecorded postcodes in the process


test the integrity of the final result

In [22]:
msoa_desirability = final_desirability[['desirability.metric', 'MSOA Code']]
print("Dropped " + str(new_length - msoa_desirability.shape[0]) + " out of " + str(new_length) + " unrecorded postcodes in the process")
number_of_na_desirability = msoa_desirability['desirability.metric'].isna().sum()
print("Number of NA desirability : " + str(number_of_na_desirability) + " out of " + str(msoa_desirability.shape[0]) )

Dropped 0 out of 9701 unrecorded postcodes in the process
Number of NA desirability : 0 out of 9701


store intermediate values in a file in case something goes wrong (long compilation time)

# Step 3 : aggregate MSOA desirability
Take the maximum within each area

In [32]:
print(len(msoa_desirability['MSOA Code'].unique()))
msoa_desirability = msoa_desirability.groupby('MSOA Code').max()

4896


# Final step : store results in a file

In [35]:
msoa_desirability.to_csv('data/desirability_by_msoa.csv', index=True)

**TO DO :** check on a map how many msoa have a corresponding desirability score. They should almost all have one
if not, try to understand what it means (does it mean that there are no schools ? if not knn, if so, low desirability score)